<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_MLP_SHAP_whitelist_BIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load dataset & model

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [ ]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

In [ ]:
x_normal = df[df['Label'] == 0].reset_index(drop=True)

In [ ]:
x_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54797 entries, 0 to 54796
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      54797 non-null  float64
 1   Tot Fwd Pkts       54797 non-null  float64
 2   Tot Bwd Pkts       54797 non-null  float64
 3   TotLen Fwd Pkts    54797 non-null  float64
 4   TotLen Bwd Pkts    54797 non-null  float64
 5   Fwd Pkt Len Max    54797 non-null  float64
 6   Fwd Pkt Len Min    54797 non-null  float64
 7   Fwd Pkt Len Mean   54797 non-null  float64
 8   Fwd Pkt Len Std    54797 non-null  float64
 9   Bwd Pkt Len Max    54797 non-null  float64
 10  Bwd Pkt Len Min    54797 non-null  float64
 11  Bwd Pkt Len Mean   54797 non-null  float64
 12  Bwd Pkt Len Std    54797 non-null  float64
 13  Flow Byts/s        54797 non-null  float64
 14  Flow Pkts/s        54797 non-null  float64
 15  Flow IAT Mean      54797 non-null  float64
 16  Flow IAT Std       547

In [ ]:
x_normal['Label'].value_counts()

0    54797
Name: Label, dtype: int64

In [ ]:
df = df.drop(columns=['Label'])

In [ ]:
x_normal = x_normal.drop(columns=['Label'])

In [ ]:
feature_list = x_normal.columns

# SHAP

In [ ]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [ ]:
normal_1500 = x_normal.head(1500)

In [ ]:
shap_values = explainer.shap_values(normal_1500)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [36]:
data_df = pd.DataFrame(shap_values[0])

In [ ]:
data_df.to_csv('/content/drive/MyDrive/1500_samples_[0].csv', index=False)

In [37]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,-0.018844,0.000000,0.000000,0.000000,-0.027909,0.000000,0.000000,-0.010201,0.118407,...,-0.007069,0.0,0.015140,0.000000,0.030807,0.000000,0.000000,0.052903,0.000000,-0.009678
1,-0.011060,0.000000,-0.003348,-0.001725,0.004852,-0.014023,0.021402,-0.034024,0.006914,0.000000,...,-0.005075,0.0,0.000000,-0.004904,-0.001303,0.000000,0.004519,0.000000,0.007941,0.012194
2,-0.014344,0.000000,-0.001870,0.000000,0.005058,-0.018353,0.009428,-0.028708,-0.006656,0.005447,...,0.000000,0.0,0.000000,0.000000,0.005787,0.000000,0.000000,0.000000,0.000000,0.005806
3,-0.015888,-0.001440,-0.004902,0.000000,0.008830,-0.016153,0.010485,-0.019995,0.000000,0.012009,...,-0.001314,0.0,-0.006393,-0.002963,0.004646,-0.002506,-0.002316,-0.006653,-0.001433,0.005329
4,-0.002993,0.009352,0.014331,0.000000,0.000000,-0.036147,0.000000,-0.030748,-0.027657,0.000000,...,-0.006298,0.0,-0.003051,0.000000,0.000000,-0.006266,0.000000,0.011585,0.000000,0.000000


In [38]:
data_dict = {}
for idx in range(0, 1500):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Normal sample ' + str(idx)] = row_values

In [ ]:
print(data_dict)

{'Normal sample 0': ['Bwd Pkt Len Max', 'Bwd IAT Max', 'Idle Std', 'ACK Flag Cnt', 'Active Max', 'Bwd Pkt Len Std', 'Flow IAT Max', 'Active Mean', 'Bwd IAT Min', 'Flow Duration'], 'Normal sample 1': ['Down/Up Ratio', 'Fwd Pkt Len Min', 'Init Bwd Win Byts', 'Idle Min', 'Idle Max', 'Bwd Seg Size Avg', 'Bwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Fwd IAT Mean', 'Fwd Pkts/s'], 'Normal sample 2': ['Down/Up Ratio', 'Bwd Seg Size Avg', 'Init Bwd Win Byts', 'Bwd IAT Min', 'Bwd Pkt Len Mean', 'Fwd Pkt Len Min', 'Fwd Header Len', 'Idle Min', 'Active Max', 'Bwd Pkt Len Max'], 'Normal sample 3': ['Down/Up Ratio', 'Init Bwd Win Byts', 'Bwd Pkt Len Max', 'Flow IAT Std', 'Bwd Pkt Len Mean', 'Fwd Pkt Len Min', 'TotLen Bwd Pkts', 'Fwd IAT Std', 'Bwd Seg Size Avg', 'Bwd Pkt Len Std'], 'Normal sample 4': ['SYN Flag Cnt', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Flow Byts/s', 'Idle Std', 'Down/Up Ratio', 'Tot Fwd Pkts', 'Flow IAT Std', 'Bwd IAT Mean', 'TotLen Fwd Pkts'], 'Normal sample 5': ['Down/Up Ratio', 'Init B

In [39]:
len(data_dict)

1500

# New whitelist: 38 top features with over 10k freq each

In [40]:
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, Normal sample 0 to Normal sample 1499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Feature 1   1500 non-null   object
 1   Feature 2   1500 non-null   object
 2   Feature 3   1500 non-null   object
 3   Feature 4   1500 non-null   object
 4   Feature 5   1500 non-null   object
 5   Feature 6   1500 non-null   object
 6   Feature 7   1500 non-null   object
 7   Feature 8   1500 non-null   object
 8   Feature 9   1500 non-null   object
 9   Feature 10  1500 non-null   object
dtypes: object(10)
memory usage: 128.9+ KB


In [42]:
df.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10
Normal sample 0,Bwd Pkt Len Max,Bwd IAT Max,Idle Std,ACK Flag Cnt,Active Max,Bwd Pkt Len Std,Flow IAT Max,Active Mean,Bwd IAT Min,Flow Duration
Normal sample 1,Down/Up Ratio,Fwd Pkt Len Min,Init Bwd Win Byts,Idle Min,Idle Max,Bwd Seg Size Avg,Bwd Pkt Len Mean,Fwd Pkt Len Std,Fwd IAT Mean,Fwd Pkts/s
Normal sample 2,Down/Up Ratio,Bwd Seg Size Avg,Init Bwd Win Byts,Bwd IAT Min,Bwd Pkt Len Mean,Fwd Pkt Len Min,Fwd Header Len,Idle Min,Active Max,Bwd Pkt Len Max
Normal sample 3,Down/Up Ratio,Init Bwd Win Byts,Bwd Pkt Len Max,Flow IAT Std,Bwd Pkt Len Mean,Fwd Pkt Len Min,TotLen Bwd Pkts,Fwd IAT Std,Bwd Seg Size Avg,Bwd Pkt Len Std
Normal sample 4,SYN Flag Cnt,Bwd Pkt Len Std,Tot Bwd Pkts,Flow Byts/s,Idle Std,Down/Up Ratio,Tot Fwd Pkts,Flow IAT Std,Bwd IAT Mean,TotLen Fwd Pkts


In [53]:
df.to_csv('/content/drive/MyDrive/MLP_top10_of_each_normalfile.csv')

In [43]:
freq = {}
for column in df.columns:
  for feature in df[column]:
    if feature in freq:
      continue
    else:
      count = sum(df[column_2] == feature for column_2 in df.columns)
      freq[feature] = count.sum()
      print(freq[feature])

581
874
293
405
377
251
69
780
183
168
124
3
841
14
285
455
746
137
764
252
206
357
142
163
175
143
185
145
291
178
258
166
147
200
175
165
223
67
366
130
195
186
163
142
228
508
125
133
150
159
130
130
174
46
62
124
149
246
85
47
97
73
79
83
2


In [44]:
len(freq)

65

In [45]:
freq_df = pd.DataFrame(list(freq.items()), columns=['Feature', 'Frequency'])
freq_df.head()

,Feature,Frequency
0,Bwd Pkt Len Max,581
1,Down/Up Ratio,874
2,SYN Flag Cnt,293
3,ACK Flag Cnt,405
4,Flow Duration,377


In [46]:
freq_df.sort_values(by=['Frequency'], ascending=False, inplace=True)
freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 1 to 64
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feature    65 non-null     object
 1   Frequency  65 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [82]:
freq_df.head(60)

,Feature,Frequency
1,Down/Up Ratio,874
12,Bwd Pkt Len Mean,841
7,Init Bwd Win Byts,780
18,Bwd Seg Size Avg,764
16,Fwd Pkt Len Min,746
0,Bwd Pkt Len Max,581
45,Idle Min,508
15,Bwd Pkt Len Std,455
3,ACK Flag Cnt,405
4,Flow Duration,377


In [52]:
freq_df.to_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv', index=False)

# Detect normal samples

In [75]:
whitelist = freq_df.iloc[:, 0][:50].tolist()
print(whitelist)

['Down/Up Ratio', 'Bwd Pkt Len Mean', 'Init Bwd Win Byts', 'Bwd Seg Size Avg', 'Fwd Pkt Len Min', 'Bwd Pkt Len Max', 'Idle Min', 'Bwd Pkt Len Std', 'ACK Flag Cnt', 'Flow Duration', 'Flow IAT Std', 'Idle Mean', 'SYN Flag Cnt', 'Bwd IAT Std', 'Tot Bwd Pkts', 'Tot Fwd Pkts', 'PSH Flag Cnt', 'Bwd IAT Max', 'TotLen Bwd Pkts', 'Fwd IAT Tot', 'TotLen Fwd Pkts', 'Bwd IAT Tot', 'Bwd Pkts/s', 'Idle Max', 'Flow IAT Mean', 'Bwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Bwd IAT Min', 'Flow IAT Max', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min', 'Active Min', 'Fwd IAT Std', 'Subflow Bwd Pkts', 'Active Max', 'Pkt Len Max', 'Active Std', 'Active Mean', 'Pkt Len Std', 'Flow IAT Min', 'Pkt Len Min', 'Fwd IAT Max', 'Pkt Len Mean', 'Fwd IAT Min', 'Subflow Bwd Byts', 'Pkt Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Pkt Len Var']


In [76]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:719


In [80]:
whitelist = freq_df.iloc[:, 0][:60].tolist()
print(whitelist)

['Down/Up Ratio', 'Bwd Pkt Len Mean', 'Init Bwd Win Byts', 'Bwd Seg Size Avg', 'Fwd Pkt Len Min', 'Bwd Pkt Len Max', 'Idle Min', 'Bwd Pkt Len Std', 'ACK Flag Cnt', 'Flow Duration', 'Flow IAT Std', 'Idle Mean', 'SYN Flag Cnt', 'Bwd IAT Std', 'Tot Bwd Pkts', 'Tot Fwd Pkts', 'PSH Flag Cnt', 'Bwd IAT Max', 'TotLen Bwd Pkts', 'Fwd IAT Tot', 'TotLen Fwd Pkts', 'Bwd IAT Tot', 'Bwd Pkts/s', 'Idle Max', 'Flow IAT Mean', 'Bwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Bwd IAT Min', 'Flow IAT Max', 'Fwd Pkt Len Std', 'Bwd Pkt Len Min', 'Active Min', 'Fwd IAT Std', 'Subflow Bwd Pkts', 'Active Max', 'Pkt Len Max', 'Active Std', 'Active Mean', 'Pkt Len Std', 'Flow IAT Min', 'Pkt Len Min', 'Fwd IAT Max', 'Pkt Len Mean', 'Fwd IAT Min', 'Subflow Bwd Byts', 'Pkt Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Pkt Len Var', 'Bwd IAT Mean', 'Fwd Pkts/s', 'Fwd IAT Mean', 'Flow Byts/s', 'Fwd Header Len', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'FIN Flag Cnt', 'Fwd Seg Size Avg', 'URG Flag Cnt']


In [81]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:111


## False positive of XAI on normal samples of trainset:
top 50: 719/1500

top 60: 111/1500